In [1]:
!pip install deviantart --upgrade

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for deviantart, since package 'wheel' is not installed.
Using legacy 'setup.py install' for sanction, since package 'wheel' is not installed.
  Running setup.py install for sanction ... done
  Running setup.py install for deviantart ... done


In [2]:
!pip install requests_cache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 10.3 MB/s eta 0:00:00


In [25]:
!pip install pandas

In [36]:
import re
import requests_cache
import deviantart
from bs4 import BeautifulSoup
import requests
import re, time, random
import pandas as pd
import io
import os, json
#from google.colab import files

In [47]:
class DeviantArtScraper:
    def __init__(self, client_id, client_secret):
        self.da = deviantart.Api(client_id, client_secret)

    def scrape_artwork_page(self,url):
        response = requests.get(url)
        artWrk_soup = BeautifulSoup(response.content, "html.parser")
        main_div = artWrk_soup.select_one("body > main > div")  # Example selector

        sub_divs = main_div.find_all("div") 
        for sub_div in sub_divs:
            text = sub_div.text.strip()
            # Extract all text from the artwork page
        #all_text = artWrk_soup.get_text(strip=True)  # or target specific elements
        return all_text
    
    def scrape(self,username):   
        page_no = 1
        gallery_data = []              
        try:
                # --- About Page ---
                about_page_url = f"https://www.deviantart.com/{username}/about"
                about_page_response = requests.get(about_page_url)
                about_page_response.raise_for_status()  # Raise an exception for bad responses
          
                about_page_soup = BeautifulSoup(about_page_response.content, "html.parser")
                id_pattern = re.compile(r"module-\d{10}")
                # Extract About page (adjust the selector based on the page structure)
                aboutpage = about_page_soup.select_one("div.wuU4s._3tnu8._2yPuh")
                #userstats_element = about_page_soup.find(id="userstats")  # Userstats
                #watchers_element = about_page_soup.find(id="watchers")
                #watching_element = about_page_soup.find(id="watching")
                group_element = about_page_soup.find(id="group_list_members")
                badges_element = about_page_soup.find(id="badges_activity")
                about_deviant = about_page_soup.find(id=id_pattern)
                
                #userstats = userstats_element.text.strip() if userstats_element else ""
                #watchers = watchers_element.text.strip() if watchers_element else ""
                #watching = watching_element.text.strip() if watching_element else ""
                groupmem = group_element.text.strip() if group_element else ""
                badges = badges_element.text.strip() if badges_element else ""
                about_devts = about_deviant.text.strip().split() if about_deviant else []            
                # --- Gallery ---
                while True:
                    if page_no == 1:
                        gallery_url = f"https://www.deviantart.com/{username}/gallery/"
                    else:
                        gallery_url = f"https://www.deviantart.com/{username}/gallery/?page={page_no}"
      
                    gallery_response = requests.get(gallery_url)
                    gallery_response.raise_for_status()
        
                    gallery_soup = BeautifulSoup(gallery_response.content, "html.parser")
        
                    # Extract gallery items (adjust the selector based on the page structure)
                    # Find all artwork links with "art" followed by anything
                    artwork_links = gallery_soup.find_all("a", href=lambda href: href and href.startswith("/art/") )
                    for link in artwork_links:
                        artwork_url = "https://www.deviantart.com" + link["href"]
                        deviation_id = artwork_url.split("/")[-1]  # Extract deviation ID
                        deviation_text = scrape_artwork_page(artwork_url)  
                        gallery_data.append([deviation_id, text])  # Append data to the list

                    next_page_link = gallery_soup.find("a", rel="next")  # Adjust selector if needed
                    if not next_page_link:
                        break  # Exit the loop if there's no next page link
                
                    page_no+=1


            # Create a Pandas DataFrame

                return groupmem, badges, about_devts #, gallery_data
                gallery_data = []
    
            
        except requests.exceptions.RequestException as e:
                print(f"Error scraping DeviantArt profile: {e}")

In [48]:
scraper = DeviantArtScraper("42096", "97080792c6d30a4178965e41f1ca15de")

# Read usernames from CSV
usernames_df = pd.read_csv("/mnt/hdd/maittewa/deviants_profileRandomWalkerSince2003.csv")  # Replace with your CSV file

# Keep track of already scraped usernames
scraped_usernames = set()
if os.path.exists("/mnt/hdd/maittewa/deviants_bio_grpmem_tags_RandomWalkedSince2003.csv"):
    existing_df = pd.read_csv("/mnt/hdd/maittewa/deviants_bio_grpmem_tags_RandomWalkedSince2003.csv")
    scraped_usernames.update(set(existing_df['username'].tolist()))

# Initialize an empty list to store scraped data
all_about_data = []

batch_size = 10
for i in range(23569, len(usernames_df), batch_size):
    batch_usernames = usernames_df['user'][i : i + batch_size].tolist()

    for username in batch_usernames:
        if username not in scraped_usernames:
          try:
            scrape_result = scraper.scrape(username)
                #watchers, watching = scraper.get_watchers_and_watching(username)
            if scrape_result:
                groupmem, badges, about_info = scrape_result #, dvtn_data
            else:
                print(f"Skipping {username} due to scraping error")
                continue

            user_data = {
                    "username": username,
                    "about_info": about_info,
                    "group_members_of": groupmem,
                    "Badges": badges,
                    #"Scraped_deviation_metadata": dvtn_data
                }
            print(f"Successfully processed user: {username}")  # Print here

            if user_data:
                all_about_data.append(user_data)  # Append to list
                scraped_usernames.add(username)  # Mark as scraped

            time.sleep(random.uniform(5, 15))  # Sleep after each user
          except Exception as e:
                print(f"Error processing user {username}: {e}")
                # Consider logging the error or taking other actions
                time.sleep(random.uniform(5, 15))  # Sleep to avoid potential rate limits


    # Create a DataFrame from the current batch
    batch_df = pd.DataFrame(all_about_data)
    # Save the batch DataFrame to CSV (append if file exists)
    batch_df.to_csv("/mnt/hdd/maittewa/deviants_bio_grpmem_tags_Dvtn_metadataRandomWalkedSince2003.csv", mode="a", header=not os.path.exists("/mnt/hdd/maittewa/deviants_bio_grpmem_tags_RandomWalkedSince2003.csv"), index=False)
#    deviation_meta.to_csv("/mnt/hdd/maittewa/scraped_deviatns_metadata_RandomWalkedSince2003.csv", mode="a", header=not os.path.exists("/mnt/hdd/maittewa/scraped_deviatns_metadata_RandomWalkedSince2003.csv"), index=False)
    # Clear the list for the next batch
    all_about_data = []

    print(f"Batch {i // batch_size + 1} completed and saved.")
    time.sleep(5)  # Sleep to avoid potential rate limits

print("Scraping completed.")

Batch 2357 completed and saved.
Batch 2358 completed and saved.
Batch 2359 completed and saved.
Successfully processed user: Personghost
Successfully processed user: reapersun
Successfully processed user: BaselMahmoud
Successfully processed user: margarita-morrigan
Successfully processed user: PemaMendez
Batch 2360 completed and saved.
Successfully processed user: VeroNArt
Successfully processed user: itsbenoclock
Successfully processed user: Byassu
Successfully processed user: kamuikaoru
Successfully processed user: lituhayu
Successfully processed user: CypherVisor
Successfully processed user: Spikie
Successfully processed user: marcelflisiuk
Successfully processed user: WhitePlumFragrance
Successfully processed user: NinaBolenArt
Batch 2361 completed and saved.
Successfully processed user: AradiyaToys
Successfully processed user: MalariaArt
Successfully processed user: FortunataFox
Successfully processed user: moonfreak
Successfully processed user: giantoothpick
Successfully processe

In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_artwork_page(url):
        response = requests.get(url)
        artWrk_soup = BeautifulSoup(response.content, "html.parser")
        main_div = artWrk_soup.select_one("body > main > div")  # Example selector

        sub_divs = main_div.find_all("div") 
        all_text = []
        if sub_divs is not None:
            for sub_div in sub_divs:
                text = sub_div.text.strip()
                all_text.append(text)
                # Extract all text from the artwork page
            #all_text = artWrk_soup.get_text(strip=True)  # or target specific elements
            return all_text
        else:
            print("No sub divs found")
    
def scrape_gallery(base_url):
    page_num = 1
    data = []  # Initialize an empty list to store the data

    while True:
        if page_num == 1:
            url = base_url
        else:
            url = f"{base_url}?page={page_num}"

        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        artwork_links = soup.find_all("a", href=lambda href: href and "/art/" in href)
        for link in artwork_links:
            artwork_url = link["href"]
            #print(artwork_url)
            deviation_id = artwork_url.split("/")[-1]  # Extract deviation ID
            text = scrape_artwork_page(artwork_url)  
            #print (text) 
            data.append([deviation_id, text])  # Append data to the list

        next_page_link = soup.select_one("#sub-folder-gallery > div > div._1DwAZ > div > a:nth-child(2)")
        if next_page_link:
            next_page_url = "https://www.deviantart.com" + next_page_link["href"]
            print(next_page_url)
        else:
            break  # Exit the loop if there's no next page link

        page_num += 1

    # Create a Pandas DataFrame
    df = pd.DataFrame(data, columns=["Deviation ID", "Text"])
    df.to_csv("deviantart_data.csv", index=False)  # Save to CSV file
    print("Data saved to deviantart_data.csv")


# Start scraping
base_url = "https://www.deviantart.com/latt-la/gallery"
scrape_gallery(base_url)

https://www.deviantart.comhttp://www.deviantart.com/latt-la/gallery?page=2
https://www.deviantart.comhttp://www.deviantart.com/latt-la/gallery
https://www.deviantart.comhttp://www.deviantart.com/latt-la/gallery
https://www.deviantart.comhttp://www.deviantart.com/latt-la/gallery
https://www.deviantart.comhttp://www.deviantart.com/latt-la/gallery


AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
https://www.deviantart.com/latt-la/art/272-Cm-Zephyr-1149346262

In [8]:
usernames_df = pd.read_csv("/mnt/hdd/maittewa/deviants_profileRandomWalkerSince2003.csv")
usernames_df.user.value_counts()

user
mjdaluz           6
NibelArt          6
Stridsberg        5
Zezhou            5
Raindropmemory    5
                 ..
Toonicorn         1
kampfschlumpf     1
apricotjackal     1
Firequill         1
Graemejukes       1
Name: count, Length: 21462, dtype: int64

In [6]:
usernames_df.loc[usernames_df.user == "Pink-Thistle"]

,user,real_name,profil_url,tags,country,bio,level,specialty,user_deviations,user_favourites,user_comments,profile_pageviews,profile_comments
23568,Pink-Thistle,NaN,https://www.deviantart.com/pink-thistle,NaN,223,NaN,NaN,NaN,59,2216,199,14011,165


In [ ]:
df = pd.read_csv("/mnt/hdd/maittewa/deviants_bio_grpmem_tags_RandomWalkedSince2003.csv")

In [2]:
df = pd.read_csv('/mnt/hdd/maittewa/deviants60K_2fin.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53598 entries, 0 to 53597
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   username          53598 non-null  object
 1   statistics        53598 non-null  object
 2   about_info        53598 non-null  object
 3   group_members_of  24739 non-null  object
 4   Badges            52698 non-null  object
dtypes: object(5)
memory usage: 2.0+ MB


In [4]:
df.tail()

,username,statistics,about_info,group_members_of,Badges
53593,FotosUrbanizadas,"{'Pageviews': 18, 'Deviations': 6, 'Watchers':...","['FotosUrbanizadasArtist', '//', 'Hobbyist', '...",NaN,Badges5 Badges3Llama1Baby Blob1Bouncing BlobGi...
53594,starrrrdog,"{'Pageviews': 37, 'Deviations': 34, 'Watchers'...","['starrrrdogUnited', 'KingdomDeviant', 'for', ...",NaN,Badges10 Badges1Mystery Egg6Llama1Baby Blob1Bo...
53595,CryoakHavenAdmin,"{'Pageviews': 126, 'Deviations': 92, 'Watchers...","['CryoakHavenAdminDeviant', 'for', '1', 'month...",NaN,Badges12 Badges1Mystery Egg1Baby Blob6Llama2Qu...
53596,charazm,"{'Pageviews': 35, 'Deviations': 4, 'Watchers':...","['charazmMay', '6United', 'StatesDeviant', 'fo...",NaN,Badges5 Badges1Mystery Egg3Llama1Baby BlobGive...
53597,CosmicMemoria,"{'Pageviews': 45, 'Deviations': 6, 'Watchers':...","['CosmicMemoriaJuly', '23Deviant', 'for', '4',...",NaN,Badges7 Badges1Mystery Egg5Llama1Bouncing Blob...


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_devart_gallery_meta():
    """
    Scrapes DeviantArt galleries for deviants listed in a DataFrame column.

    Args:
        df: The pandas DataFrame containing the deviant usernames.
        deviant_column_name: The name of the column containing the usernames.

    Returns:
        A pandas DataFrame containing the scraped data.
    """

    all_deviation_data = []
    usernames = set(df[deviant_column_name].tolist())
    for deviant_username in usernames:
        page_no = 1
        while true:
            gallery_url = f"https://www.deviantart.com/https:/leork-dream/gallery/all/page={page_no}"
            
            try:
                response = requests.get(gallery_url)
                response.raise_for_status()  # Raise an exception for bad responses
    
                soup = BeautifulSoup(response.content, "html.parser")
    
                # Find the target div containing the deviation data
                deviation_divs = soup.find_all("div", class_="_3mrQ-. _1h0V1")  # Adjust class if needed
    
                for deviation_div in deviation_divs:
                    deviation_data = {}
                    
                    # Example: Extract data from inner divs within deviation_div
                    # Example div structure:
                    # <div class="deviation">
                    #   <div class="title">...</div>
                    #   <div class="description">...</div>
                    #   <div class="tags">...</div>
                    # </div>
                    
                    dev_fav = deviation_div.find("div", class_="_2kMWf _3zzGu")  
                    if title_div:
                        deviation_data["Favourites"] = dev_fav.text.strip()
    
                    tags_div = deviation_div.find("div", class_="_3jkpi _2-cqb")  
                    if tags_div:
                        deviation_data["Tags"] = [tag.text.strip() for tag in tags_div.find_all("a")]
                    
                    createdBy_div = deviation_div.find("div", class_="_2xsLm")  
                    if createdBy_div:
                        deviation_data["CreatedBy"] = createdBy.text.strip()
    
                    # ... Extract other data as needed ...
    
                    all_deviation_data.append(deviation_data)
                 page_no += 1  # Move to the next page
    
            except requests.exceptions.RequestException as e:
                print(f"Error scraping gallery for {deviant_username}: {e}")
    
        scraped_gallery_info = pd.DataFrame(all_deviation_data)
        return scraped_gallery_info
        
    
    # Example usage:
    # Assuming your DataFrame is called 'deviants_df' and the column with usernames is 'username'

In [ ]:
scraped_data_df = scrape_deviantart_gallery()
print(scraped_data_df)

In [ ]:
body > main > div > div._3bEH0._1ztk6 > div

In [1]:
import pandas as pd

In [2]:
pd_gall_dat = pd.read_csv("/mnt/hdd/maittewa/deviant_gallery_dataRandomWalkerSince2003.csv")
#pd_devmeta_dat = pd.read_csv("/mnt/hdd/maittewa/deviants_deviationMetaDataRandomWalkerSince2003.csv")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
pd_gall_dat.info()

In [ ]:
pd_gall_dat.Deviation_id.value_counts()

In [7]:
pd_gall_data_withoutDuplicates = pd_gall_dat.drop_duplicates(subset=['Deviation_id'], keep='first', inplace = False)
#df.drop_duplicates(subset=['column_name'], keep='first', inplace=True)


In [8]:
pd_gall_data_withoutDuplicates.to_csv("/mnt/hdd/maittewa/uniqueDeviant_gallery_dataRandomWalkerSince2003.csv")

In [37]:
dev_df = pd.read_csv("/mnt/hdd/maittewa/deviants_profileRandomWalkerSince2003.csv")
dev_df.loc[dev_df.user == "4130013"]

,user,real_name,profil_url,tags,country,bio,level,specialty,user_deviations,user_favourites,user_comments,profile_pageviews,profile_comments
1779,4130013,NaN,https://www.deviantart.com/4130013,sweetcandy(紫),46,NaN,NaN,NaN,162,645,578,23191,280


In [23]:
dev_df.user.value_counts()

user
mjdaluz          6
NibelArt         6
sanoe            5
Quirkilicious    5
silwenka         5
                ..
Tsubaki-art      1
Relative         1
mtmac            1
julishland       1
Akseru           1
Name: count, Length: 21965, dtype: int64